In [11]:
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
from stocks import Stocks, Ticker

In [12]:
stocks = Stocks()

Retrieved current S&P500 tickers ['MMM', 'AOS', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA

 44%|███████████████████████████████████████████▌                                                        | 219/503 [00:00<00:00, 2188.14it/s]

Setting up MMM
Setting up AOS
Setting up ABT
Setting up ABBV
Setting up ABMD
Setting up ACN
Setting up ATVI
Setting up ADM
Setting up ADBE
Setting up ADP
Setting up AAP
Setting up AES
Setting up AFL
Setting up A
Setting up APD
Setting up AKAM
Setting up ALK
Setting up ALB
Setting up ARE
Setting up ALGN
Setting up ALLE
Setting up LNT
Setting up ALL
Setting up GOOGL
Setting up GOOG
Setting up MO
Setting up AMZN
Setting up AMCR
Setting up AMD
Setting up AEE
Setting up AAL
Setting up AEP
Setting up AXP
Setting up AIG
Setting up AMT
Setting up AWK
Setting up AMP
Setting up ABC
Setting up AME
Setting up AMGN
Setting up APH
Setting up ADI
Setting up ANSS
Setting up AON
Setting up APA
Setting up AAPL
Setting up AMAT
Setting up APTV
Setting up ANET
Setting up AJG
Setting up AIZ
Setting up T
Setting up ATO
Setting up ADSK
Setting up AZO
Setting up AVB
Setting up AVY
Setting up BKR
Setting up BALL
Setting up BAC
Setting up BBWI
Setting up BAX
Setting up BDX
Setting up WRB
Setting up BRK.B
Setting

 92%|███████████████████████████████████████████████████████████████████████████████████████████▊        | 462/503 [00:00<00:00, 2327.74it/s]

Setting up UAL
Setting up UPS


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 503/503 [00:00<00:00, 2302.54it/s]

Setting up URI
Setting up UNH
Setting up UHS
Setting up VLO
Setting up VTR
Setting up VRSN
Setting up VRSK
Setting up VZ
Setting up VRTX
Setting up VFC
Setting up VTRS
Setting up VICI
Setting up V
Setting up VNO
Setting up VMC
Setting up WAB
Setting up WBA
Setting up WMT
Setting up WBD
Setting up WM
Setting up WAT
Setting up WEC
Setting up WFC
Setting up WELL
Setting up WST
Setting up WDC
Setting up WRK
Setting up WY
Setting up WHR
Setting up WMB
Setting up WTW
Setting up GWW
Setting up WYNN
Setting up XEL
Setting up XYL
Setting up YUM
Setting up ZBRA
Setting up ZBH
Setting up ZION
Setting up ZTS


In [6]:
# Size of dataset
size = 100

# Predictor variable
price = np.random.randn(size)

In [7]:
model = pm.Model()

with model:
    # Priors for unknown model parameters
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10, shape=2)
    sigma = pm.HalfNormal("sigma", sigma=1)

